In [ ]:
# import all needed modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import glob

In [ ]:
# define function for vibration partition function calculator
def qvib_calc(vibs,T):
    kB = 8.617333262145*10**(-5) # eV/K, Boltzmann Constant
    qvib = 1
    for item in vibs:
        if item != 0:
            temp = np.exp(-item/2/kB/T/1000)/(1 - np.exp(-item/kB/T/1000))
            qvib = qvib*temp
    return qvib

In [ ]:
# define function for reading in and parsing vibrational modes
def vib_read(path_to_file,filename): #filename contains all vibs for H, O, and OH on (111), (100), and (110)
    f = open(path_to_file+'/'+filename,"r")
    lines = f.readlines()
    vibs = []
    for ii in range(1,len(lines)):
        line_split = lines[ii].split()
        for item in line_split:
            vibs.append(float(item))
    return vibs

In [ ]:
# define function for reading in and parsing energies
# all energies for clean, H, O, and OH on (111), (100), and (110) with promoter in surface and subsurface
def energy_read(path_to_file,filename): 
    f = open(path_to_file+'/'+filename,"r")
    lines = f.readlines()
    N = int((len(lines)-1)/3)
    en_surf = []
    en_sub = []
    for ii in range(0,N):
        t1 = []
        t2 = []
        for jj in range(1,len(lines),N):
            line_split = lines[ii+jj].split()
            t1.append(float(line_split[1]))
            t2.append(float(line_split[2]))
        en_surf.append(t1)
        en_sub.append(t2)
    f.close()
    return [en_surf,en_sub]

In [ ]:
# define function for performing kubic harmonic regression
def kubic_harm(prop): #order is in (111), (100), and (110)
    x1 = [0.333333333,1,0.5]
    x2 = [0.037037037,0,0]
    fit_x = np.array([x1,x2]).transpose()
    model = LinearRegression()
    model.fit(fit_x,prop)
    return [model.intercept_,model.coef_[0],model.coef_[1]]

In [ ]:
# define function for calculating vibrational partition function kubic harmonic fit at set tmperature
def KH_qvib(vibs1,vibs2,vibs3,T): #order is in (111), (100), and (110)
    qvib = [qvib_calc(vibs1,T),qvib_calc(vibs2,T),qvib_calc(vibs3,T)]
    [qv0,qv4,qv6] = kubic_harm(qvib)
    return [qv0,qv4,qv6]

In [ ]:
def NP_plot(ph1,ph2,Z,sys_name,p,surf_name,item,scale_min,scale_max,dpi_set,colmap):
    fig, ax = plt.subplots(subplot_kw=dict(projection='polar'),figsize=(9,8))
    im = ax.contourf(ph1,ph2,Z,vmin=scale_min,vmax=scale_max,cmap=colmap)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    plt.setp(ax.spines.values(), linewidth=1.5)
    fig.colorbar(im)
    ax.set_xticks([])
    ax.set_yticks([])
    fig.set(facecolor='white')
    fig.savefig(p+surf_name+sys_name+str(item)+".png",dpi=dpi_set)
    plt.close(fig)

In [ ]:
# define function for calculating equilibrium coverages over NP model
def NP_calc(npoints,T,P0,yH2,yO2,E_kub,QV_kub,Ef_kub):
    phi = np.radians(np.linspace(0,360,npoints))
    theta = np.radians(np.linspace(0,90,npoints))
    
    E_H = E_kub[0]
    E_O = E_kub[1]
    E_OH = E_kub[2]
    
    QV_H = QV_kub[0]
    QV_O = QV_kub[1]
    QV_OH = QV_kub[2]
    
    Ef_S = Ef_kub[0] 
    Ef_H = Ef_kub[1]
    Ef_O = Ef_kub[2]
    Ef_OH = Ef_kub[3]
    
    # Calculate in situ partial pressures based on initial H2 and O2 partial pressure and assuming reaction reaches 99% of equilibrium
    P0_H2 = yH2*P0 # Pa, initial H2 partial pressures
    P0_O2 = yO2*P0 # Pa, initial O2 partial pressures
    P_inert = (1 - yH2 - yO2)*P0 # Pa, accounts for any inert gas species present
    ext_rxn = min(2*P0_O2,P0_H2)*0.99 # Pa, extent of reaction assumed to reaction near 99% of equilibrium 
    P_H2 = (P0_H2 - ext_rxn)/(P0_H2 + P0_O2 + P_inert - 1/2*ext_rxn)*P0 # Pa, near equil. H2 partial pressure
    P_O2 = (P0_O2 - 1/2*ext_rxn)/(P0_H2 + P0_O2 + P_inert - 1/2*ext_rxn)*P0 # Pa, near equil. O2 partial pressure
    P_H2O = (ext_rxn)/(P0_H2 + P0_O2 + P_inert - 1/2*ext_rxn)*P0 # Pa, near equil. H2O partial pressure
    
    kB = 1.38064852*10**(-23) # J/K, Boltzmann Constant
    h = 6.62607004*10**(-34) # J*s, Plank's Constant
    mH2 = 1.00784*2/(6.022*10**23)/1000 # kg, H2 Mass
    mO2 = 31.999/(6.022*10**23)/1000 # kg, O2 Mass
    mH2O = 18.01528/(6.022*10**23)/1000 # kg, H2O Mass
    kB2 = 8.617333262145*10**(-5) # eV/K, Boltzmann Constant
    vH2 = [537.767435] # meV, H2 bond frequency in gas phase
    vO2 = [191.125756] # meV, O2 bond frequency in gas phase
    vH2O = [473.441566,459.393964,198.081414] # meV, H2O bond freqency in gas phase
    I_H2 = mH2/4*(0.747277565*10**(-10))**2 # kg*m^2, H2 moment of inertia
    I_O2 = mO2/4*(1.239620627*10**(-10))**2 # kg*m^2, O2 moment of inertia
    I_H2O = 6.3848765503633E-141 # kg^3*m^6, H2O moment of inertia
    lamb_H = h/(2*np.pi*mH2*kB*T)**(1/2)
    lamb_O = h/(2*np.pi*mO2*kB*T)**(1/2)
    lamb_H2O = h/(2*np.pi*mH2O*kB*T)**(1/2)
    
    Zrot_H2 = (8*np.pi**2*I_H2*kB*T/2/h**2) # H2 rotational partition function
    Zrot_O2 = (8*np.pi**2*I_O2*kB*T/2/h**2) # O2 rotational partition function
    Zrot_H2O = 8*np.pi**2/(2*h**3)*(2*np.pi*kB*T)**(3/2)*I_H2O**(1/2) # H2O rotational partition function
    Zvib_H2 = qvib_calc(vH2,T) # H2 vibrational partition function
    Zvib_O2 = qvib_calc(vO2,T) # O2 vibrational partition function
    Zvib_H2O = qvib_calc(vH2O,T) # O2 vibrational partition function
    Ztrans_H2 = kB*T/P_H2*1/lamb_H**3 # H2 translational partition function
    Ztrans_O2 = kB*T/P_O2*1/lamb_O**3 # O2 translational partition function
    Ztrans_H2O = kB*T/P_H2O*1/lamb_H2O**3 # H2O translational partition function
    
    covH = []
    covO = []
    covOH = []
    Aform_H = []
    Aform_O = []
    Aform_OH = []
    Aform_S = []
    TOF = []
    for ii in range(0,npoints): # loop over all theta values
        for jj in range(0,npoints): # loop over all phi values        
            nx = np.sin(theta[ii])*np.cos(phi[jj])
            ny = np.sin(theta[ii])*np.sin(phi[jj])
            nz = np.cos(theta[ii])
            
            # calculate Eads, coverages, etc.
            ## Eads and Erxn values
            Eads_H = E_H[0] + E_H[1]*(nx**4+ny**4+nz**4) + E_H[2]*(nx**2*ny**2*nz**2) 
            Eads_O = E_O[0] + E_O[1]*(nx**4+ny**4+nz**4) + E_O[2]*(nx**2*ny**2*nz**2)
            Erxn_OH = E_OH[0] + E_OH[1]*(nx**4+ny**4+nz**4) + E_OH[2]*(nx**2*ny**2*nz**2)
            ## vibrational partition function values            
            qv_H = QV_H[0] + QV_H[1]*(nx**4+ny**4+nz**4) + QV_H[2]*(nx**2*ny**2*nz**2) 
            qv_O = QV_O[0] + QV_O[1]*(nx**4+ny**4+nz**4) + QV_O[2]*(nx**2*ny**2*nz**2) 
            qv_OH = QV_OH[0] + QV_OH[1]*(nx**4+ny**4+nz**4) + QV_OH[2]*(nx**2*ny**2*nz**2) 
            ## equilibrium constants       
            Ke_H = (qv_H)**2/(Zrot_H2*Zvib_H2*Ztrans_H2)*np.exp(-2*Eads_H/kB2/T)
            Ke_O = (qv_O)**2/(Zrot_O2*Zvib_O2*Ztrans_O2)*np.exp(-2*Eads_O/kB2/T)
            Ke_OH = (qv_OH)/(qv_O*qv_H)*np.exp(-Erxn_OH/kB2/T)
            ## equilibrium coverages
            tS = (1 + (Ke_H*P_H2)**(1/2) + (Ke_O*P_O2)**(1/2) + Ke_OH*(Ke_H*P_H2*Ke_O*P_O2)**(1/2))**(-1)
            tH = (Ke_H*P_H2)**(1/2)*tS
            tO = (Ke_O*P_O2)**(1/2)*tS
            tOH = Ke_OH*(Ke_H*P_H2*Ke_O*P_O2)**(1/2)*tS
            ## Aform values
            Ef_H_int = Ef_H[0] + Ef_H[1]*(nx**4+ny**4+nz**4) + Ef_H[2]*(nx**2*ny**2*nz**2)
            Ef_O_int = Ef_O[0] + Ef_O[1]*(nx**4+ny**4+nz**4) + Ef_O[2]*(nx**2*ny**2*nz**2)
            Ef_OH_int = Ef_OH[0] + Ef_OH[1]*(nx**4+ny**4+nz**4) + Ef_OH[2]*(nx**2*ny**2*nz**2)
            Ef_S_int = Ef_S[0] + Ef_S[1]*(nx**4+ny**4+nz**4) + Ef_S[2]*(nx**2*ny**2*nz**2)          
            
            Af_H = Ef_H_int - kB2*T*np.log((qv_H)/(Zrot_H2*Zvib_H2*Ztrans_H2)**(1/2))
            Af_O = Ef_O_int - kB2*T*np.log((qv_O)/(Zrot_O2*Zvib_O2*Ztrans_O2)**(1/2))
            Af_OH = Ef_OH_int - kB2*T*np.log((qv_OH)/(Zrot_O2*Zvib_O2*Ztrans_O2*Zrot_H2*Zvib_H2*Ztrans_H2)**(1/2))
            Af_S = Ef_S_int
            ## TOF calculation
            E_OHwH = (Erxn_OH + Eads_O + Eads_H) + Eads_H + 2.3916165
            Eact = 0.75 - 0.55*E_OHwH
            qrat = (Zrot_H2O*Zvib_H2O*Ztrans_H2O)/(qv_OH*qv_H)
            kf = kB*T/h*qrat*np.exp(-Eact/kB2/T)
            tof = np.log10(kf*tOH*tH)
            
            covH.append(tH)
            covO.append(tO)
            covOH.append(tOH)
            Aform_H.append(Af_H)
            Aform_O.append(Af_O)
            Aform_OH.append(Af_OH)
            Aform_S.append(Af_S)
            TOF.append(tof)
    return [covH,covO,covOH,Aform_H,Aform_O,Aform_OH,Aform_S,TOF]

In [ ]:
# define function for looping over temperature for a given NiM catalyst
def TP_loops(surf_name,npoints,dpi_set):
    T = [300,475,650,825,1000] # K, user input temperature
    P0 = 1*10**5 # Pa, user input
    yH2 = 0.1 # mole fraction, user input
    yO2 = [1.0e-23,0.1] # mole fraction, user input
    phi = np.radians(np.linspace(0,360,npoints))
    ph1,ph2 = np.meshgrid(phi,phi)

    p = './'+surf_name+'/' # path to files
    fsurf_vibs = './'+surf_name+'_surf_vibs.txt' # file name for vibs with M in surface layer
    fsub_vibs = './'+surf_name+'_sub_vibs.txt' # file name for vibs with M in subsurface layer
    f_en = './'+surf_name+'_energies.txt' # file name for H2 Eads, O2 Eads, and O*+H* -> OH* Erxn
    f_Eform = './'+surf_name+'_Eform.txt' # file name for Eform under clean and adsorbate-covered cases

    # read in data from text files
    vibs_surf = vib_read(p,fsurf_vibs)
    vibs_surf = np.array(vibs_surf).reshape(6,9).transpose()
    vibs_sub = vib_read(p,fsub_vibs)
    vibs_sub = np.array(vibs_sub).reshape(6,9).transpose()
    [ens_surf,ens_sub] = energy_read(p,f_en)
    [Ef_surf,Ef_sub] = energy_read(p,f_Eform)

    # generate kubic harmonics for DFT mechanisms' Eads and Erxn 
    ens_surf_kub = []
    ens_sub_kub = []
    for ii in range(0,len(ens_surf)):
        ens_surf_kub.append(kubic_harm(ens_surf[ii]))
        ens_sub_kub.append(kubic_harm(ens_sub[ii]))

    # generate kubic harmonics for DFT Eform 
    Ef_surf_kub = []
    Ef_sub_kub = []
    for ii in range(0,len(Ef_surf)):
        Ef_surf_kub.append(kubic_harm(Ef_surf[ii]))
        Ef_sub_kub.append(kubic_harm(Ef_sub[ii]))

    # start of modeling varied properties (T,y_O2) and generating NP models
    file1 = open(p+surf_name+"_CovSum.txt", "w")
    file1.write("Temperature"+" "+"O2 fraction"+" "+"Ave. H*"+" "+"Ave. O*"+" "+"Ave. OH*"+" "+"Max. H*"+" "+"Max. O*"+" "+"Max. OH*"+" "+"Ave. M fraction"+" "+"Ave. TOF"+"\n")

    plot_cnt = 0
    # run temperature loop
    for item2 in T:
        # generate kubic harmonics for qvibs for adsorbed species
        QV_surf = []
        QV_sub = []
        for ii in range(0,3):
            QV_surf.append(KH_qvib(vibs_surf[ii],vibs_surf[ii+3],vibs_surf[ii+6],item2))
            QV_sub.append(KH_qvib(vibs_sub[ii],vibs_sub[ii+3],vibs_sub[ii+6],item2))

        # run composition loop
        for item in yO2:
            [covH_surf,covO_surf,covOH_surf,EH_surf,EO_surf,EOH_surf,ES_surf,TOF_surf] = NP_calc(npoints,item2,P0,yH2,item,
                                                                                        ens_surf_kub,QV_surf,Ef_surf_kub)

            [covH_sub,covO_sub,covOH_sub,EH_sub,EO_sub,EOH_sub,ES_sub,TOF_sub] = NP_calc(npoints,item2,P0,yH2,item,ens_sub_kub,
                                                                                 QV_sub,Ef_sub_kub)

            covH = []
            covO = []
            covOH = []
            layer = []
            TOF = []
            for ii in range(0,npoints**2):
                covS_surf = 1 - covH_surf[ii] - covO_surf[ii] - covOH_surf[ii] 
                covS_sub = 1 - covH_sub[ii] - covO_sub[ii] - covOH_sub[ii]
                Es_surf = (EH_surf[ii]*covH_surf[ii] + EO_surf[ii]*covO_surf[ii] + EOH_surf[ii]*covOH_surf[ii] 
                           + ES_surf[ii]*covS_surf)
                Es_sub = (EH_sub[ii]*covH_sub[ii] + EO_sub[ii]*covO_sub[ii] + EOH_sub[ii]*covOH_sub[ii] 
                          + ES_sub[ii]*covS_sub)
                if Es_sub < Es_surf:
                    layer.append(0)
                    covH.append(covH_sub[ii])
                    covO.append(covO_sub[ii])
                    covOH.append(covOH_sub[ii])
                    TOF.append(TOF_sub[ii])
                else:
                    layer.append(1)
                    covH.append(covH_surf[ii])
                    covO.append(covO_surf[ii])
                    covOH.append(covOH_surf[ii])
                    TOF.append(TOF_surf[ii])

            N = len(layer)
            aveH = sum(covH)/N
            aveO = sum(covO)/N
            aveOH = sum(covOH)/N
            ave_layer = sum(layer)/N*0.25
            ave_TOF = sum(TOF)/N
            file1.write(str(item2)+" "+str(item)+" "+str(aveH)+" "+str(aveO)+" "+str(aveOH)+" "+str(max(covH))+" "+str(max(covO))+" "+str(max(covOH))+" "+str(ave_layer)+" "+str(ave_TOF)+"\n")
            
            covO = np.reshape(covO,(npoints,npoints))
            covOH = np.reshape(covOH,(npoints,npoints))
            layer = np.reshape(layer,(npoints,npoints))
            TOFmin = min(TOF)
            TOFmax = max(TOF)
            covH = np.reshape(covH,(npoints,npoints))
            TOF = np.reshape(TOF,(npoints,npoints))
            plot_cnt = plot_cnt + 1
            NP_plot(ph1,ph2,covH,'CovH',p,surf_name,plot_cnt,0.0,0.98,dpi_set,'viridis')
            NP_plot(ph1,ph2,covO,'CovO',p,surf_name,plot_cnt,0.0,0.98,dpi_set,'viridis')
            NP_plot(ph1,ph2,covOH,'CovOH',p,surf_name,plot_cnt,0.0,0.98,dpi_set,'viridis')
            NP_plot(ph1,ph2,layer,'Layer',p,surf_name,plot_cnt,0.0,0.98,dpi_set,'viridis')
            NP_plot(ph1,ph2,TOF,'TOF',p,surf_name,plot_cnt,TOFmin,TOFmax,dpi_set,'RdPu_r')

            print(str(item2)+" K, "+str(item*100)+" mol% O2 completed!")
    file1.close()

In [ ]:
files = glob.glob('./Ni*/')
npoints = 100
dpi_set = 100
for item in files:
    print(item[2:6])
    TP_loops(item[2:6],npoints,dpi_set)
    print('completed!')